In [2]:
import urllib
import json
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from keys import client_id, client_secret, app_id

Datasource:
https://docs.google.com/spreadsheets/d/14quQPFErG-hlpsrNgYcX85vW7JMMK5X2vNZrafRcH8c/edit?usp=sharing

In [104]:
data = pd.read_csv('../original_datasets/tableau_time_sereis0403.csv')

In [105]:
states = data['Province_State'].unique().tolist()
dates = data['Date'].unique().tolist()
coords = data.groupby(['Lat', 'Long']).size().index.tolist()
data['avg_tmp'] = np.nan
data['avg_RH'] = np.nan
data['avg_wind'] = np.nan
data['avg_precip'] = np.nan
data['avg_m_tmp'] = np.nan
data['avg_m_RH'] = np.nan
data['avg_m_wind'] = np.nan
data['avg_m_precip'] = np.nan

# ADD CODE HERE THAT CLEARS EVERYTHING WITH LESS THEN 20 CONFIRMED
# ALSO CLEAR ROWS EXCEPT CONFIRMED
# ALSO REMOVE PLACES WHERE COORD ARE NAN
# MAKE LAT AND LONG INDEXS
# TRASNPOSE TO NORMAL TIME-SERIES

In [124]:
# set for up to 30 days a time
# START_TIME = '03/01/2020'
# END_TIME = '04/01/2020'
RADIUS = 350
INTERVAL = 4
q = 0

date_range = pd.date_range('2020-01-01', periods=100, freq='D')
start_time = date_range[0].strftime('%m/%d/%Y')

with open('weather_logs/tableau_log{0}.txt'.format(datetime.now().strftime('%d%m%Y')), 'w') as log:
    msg = 'Start date: {0}\nEnd date: {1}\nInterval:{2}'.format(start_time, date_range[-1].strftime('%m/%d/%Y'), INTERVAL)
    print(msg)

    for coord in coords:
        q += 1
        lat = coord[0]
        long = coord[1]
        location = data.loc[data['Lat'] == lat]\
                        .loc[data['Long'] == long]\
                         [['Country_Region','Province_State']]
        country = location.iloc[0][0]
        province = location.iloc[0][1]
        msg = 'collecting for {0} location {1}, {2}'.format(q, province, country)
        print(msg)
        _ = log.write(msg+'\n')
        for date_ in date_range[INTERVAL:][::INTERVAL]:
            end_time = date_.strftime('%m/%d/%Y')    
            url = 'https://api.aerisapi.com/observations/summary/closest?p={0},{1}&from={2}&to={3}&radius={4}miles&plimit=31&limit=1&'\
                  'client_id={5}&client_secret={6}'.format(lat, long, start_time, end_time, RADIUS, client_id, client_secret)
            msg = 'interval: {0}-{1}\n{2}\n'.format(start_time, end_time, url)
            print(msg)
            _ = log.write(msg+'\n')
            request = urllib.request.urlopen(url)
            response = request.read()
            json_ = json.loads(response)
            if json_['success']: 
                print('success')
                _ = log.write('success\n')
                try:
                    for day in json_['response'][0]['periods']:
                        date = datetime.strptime(str(day['summary']['ymd']), '%Y%m%d').strftime("%-m/%-d/%Y")
                        _ = log.write(date + '\n')
                        for index in data.loc[data['Date'] == date]\
                                         .loc[data['Lat'] == coord[0]]\
                                         .loc[data['Long'] == coord[1]].index:
                            data.at[index, 'avg_d_tmp'] = day['summary']['temp']['avgC']
                            data.at[index, 'avg_d_RH'] = day['summary']['rh']['avg']
                            data.at[index, 'avg_d_wind'] = day['summary']['wind']['avgKPH']
                            data.at[index, 'avg_d_precip'] = day['summary']['precip']['totalMM']
                except IndexError:
                    if json_['error']:
                        msg = 'No data found for {0}, {1} at {2}\n{3}\r\n'.format(country, province, date, json_['error'])
                        print(msg)
                        _ = log.write(msg)
                except Exception:
                    msg = 'Unknowen error\n{0}\r\n'.format(json_['error'])
                    print(msg)
                    _ = log.write(msg)
                finally:
                    request.close()
        start_time = date_ + timedelta(days=1)
        start_time = start_time.strftime('%m/%d/%Y')

Start date: 01/01/2020
End date: 04/09/2020
Interval:4
collecting for 1 location Tasmania, Australia
https://api.aerisapi.com/observations/summary/closest?p=-41.4545,145.9707&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2 location nan, New Zealand
https://api.aerisapi.com/observations/summary/closest?p=-40.9006,174.886&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 3 location nan, Argentina
https://api.aerisapi.com/observations/summary/closest?p=-38.4161,-63.6167&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 4 location Victoria, Australia
https://api.aerisapi.com/observations/summary/closest?p=-37.8136,144.96

success
collecting for 31 location Northern Territory, Australia
https://api.aerisapi.com/observations/summary/closest?p=-12.4634,130.8456&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 32 location nan, Angola
https://api.aerisapi.com/observations/summary/closest?p=-11.2027,17.8739&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 33 location nan, Peru
https://api.aerisapi.com/observations/summary/closest?p=-9.19,-75.0152&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 34 location nan, Timor-Leste
https://api.aerisapi.com/observations/summary/closest?p=-8.874217,125.727539&from=01/01/2020&to=01/05/2020&radius=350mil

success
collecting for 61 location nan, Sri Lanka
https://api.aerisapi.com/observations/summary/closest?p=7.0,81.0&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 62 location nan, Cote d'Ivoire
https://api.aerisapi.com/observations/summary/closest?p=7.54,-5.5471&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 63 location nan, Ghana
https://api.aerisapi.com/observations/summary/closest?p=7.9465,-1.0232&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 64 location nan, Sierra Leone
https://api.aerisapi.com/observations/summary/closest?p=8.460555000000001,-11.779889&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=3

success
collecting for 91 location Martinique, France
https://api.aerisapi.com/observations/summary/closest?p=14.6415,-61.0242&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 92 location nan, Thailand
https://api.aerisapi.com/observations/summary/closest?p=15.0,101.0&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 93 location Northern Mariana Islands, US
https://api.aerisapi.com/observations/summary/closest?p=15.0979,145.6739&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 94 location nan, Eritrea
https://api.aerisapi.com/observations/summary/closest?p=15.1794,39.7823&from=01/01/2020&to=01/05/2020&radius=350miles&p

success
collecting for 121 location Hawaii, US
https://api.aerisapi.com/observations/summary/closest?p=20.86399628,-156.56890969999998&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 122 location nan, Oman
https://api.aerisapi.com/observations/summary/closest?p=21.0,57.0&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 123 location nan, India
https://api.aerisapi.com/observations/summary/closest?p=21.0,78.0&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 124 location nan, Mauritania
https://api.aerisapi.com/observations/summary/closest?p=21.0079,10.9408&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1

...

success
collecting for 2549 location Indiana, US
https://api.aerisapi.com/observations/summary/closest?p=41.59761169,-85.85875695&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2550 location Ohio, US
https://api.aerisapi.com/observations/summary/closest?p=41.60213491,-84.12571393&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2551 location nan, North Macedonia
https://api.aerisapi.com/observations/summary/closest?p=41.6086,21.7453&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2552 location Nebraska, US
https://api.aerisapi.com/observations/summary/closest?p=41.61703149,-102.3313973&from=01/01/2020&to=01/05/2020

success
collecting for 2579 location Rhode Island, US
https://api.aerisapi.com/observations/summary/closest?p=41.71018079,-71.28652315&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2580 location New York, US
https://api.aerisapi.com/observations/summary/closest?p=41.71579493,-74.76394559&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2581 location Nebraska, US
https://api.aerisapi.com/observations/summary/closest?p=41.71711774,-103.0061405&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2582 location Utah, US
https://api.aerisapi.com/observations/summary/closest?p=41.72330587,-111.74436670000001&from=01/01/2020&

success
collecting for 2608 location Michigan, US
https://api.aerisapi.com/observations/summary/closest?p=41.89475156,-84.06603727&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2609 location Iowa, US
https://api.aerisapi.com/observations/summary/closest?p=41.89981633,-90.53275096&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2610 location nan, Holy See
https://api.aerisapi.com/observations/summary/closest?p=41.9029,12.4534&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2611 location Nebraska, US
https://api.aerisapi.com/observations/summary/closest?p=41.91311716,-99.97677845&from=01/01/2020&to=01/05/2020&radiu

success
collecting for 2638 location Iowa, US
https://api.aerisapi.com/observations/summary/closest?p=42.07923082,-91.59809485&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2639 location Iowa, US
https://api.aerisapi.com/observations/summary/closest?p=42.07991767,-92.53266945&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
collecting for 2640 location Iowa, US
https://api.aerisapi.com/observations/summary/closest?p=42.08011739,-92.06463573&from=01/01/2020&to=01/05/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM


KeyboardInterrupt: 

###### HANDLE NaN AND SAVE DATA

In [30]:
backup = data.copy()
# data[['Province_State']] = data[['Province_State']].fillna(0)
# data = data.drop('avg_sun', axis=1)
# data = data.dropna()

In [962]:
##############################################
# COMPUTE AVG PARAMETERS FOR CONFIRMED CASES #
##############################################

conf_data = data[data['Case_Type'] == 'Confirmed']
death_data = data[data['Case_Type'] == 'Deaths']
for coord in coords:
    lat = coord[0]
    long = coord[1]
    indices = conf_data.loc[conf_data['Lat'] == lat].loc[conf_data['Long'] == long].index.tolist()
    avg_tmp = conf_data.loc[conf_data['Lat'] == lat]\
                       .loc[conf_data['Long'] == long]['avg_tmp'].mean()
    
    avg_rh = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_RH'].mean()
    
    avg_wind = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_wind'].mean()
    
    avg_precip = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_precip'].mean()                             
    
    for index in indices:
        conf_data.at[index, 'avg_m_tmp'] = avg_tmp
        conf_data.at[index, 'avg_m_RH'] = avg_rh
        conf_data.at[index, 'avg_m_wind'] = avg_wind
        conf_data.at[index, 'avg_m_precip'] = avg_precip                            

###### COMPUTE AVG PARAMETERS FOR DEATHS CASES

In [963]:
for coord in coords:
    lat = coord[0]
    long = coord[1]
    indices = death_data.loc[data['Lat'] == lat].loc[death_data['Long'] == long].index.tolist()
    avg_tmp = death_data.loc[death_data['Lat'] == lat]\
                       .loc[death_data['Long'] == long]['avg_tmp'].mean()
    
    avg_rh = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_RH'].mean()
    
    avg_wind = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_wind'].mean()
    
    avg_precip = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_precip'].mean()                             
    
    for index in indices:
        death_data.at[index, 'avg_m_tmp'] = avg_tmp
        death_data.at[index, 'avg_m_RH'] = avg_rh
        death_data.at[index, 'avg_m_wind'] = avg_wind
        death_data.at[index, 'avg_m_precip'] = avg_precip                            

In [151]:
#########################
# This was to fix a bug #
#########################

# conf_data = pd.read_csv('../augmented_datasets/tableau_conf_data.csv')
# death_data = pd.read_csv('../augmented_datasets/tableau_death_data.csv')
# conf_data = conf_data.drop('Unnamed: 0', axis=1)
# death_data = death_data.drop('Unnamed: 0', axis=1)
# conf_data['Date'] = conf_data['Date'].apply(lambda x: x.lstrip('0'))
# death_data['Date'] = death_data['Date'].apply(lambda x: x.lstrip('0'))

##############################
# This is for multi-indexing #
##############################

# conf_data['Date'] = pd.DatetimeIndex(conf_data['Date'])
# conf_data.set_index(conf_data['Date'], append=True, inplace=True, drop=True)
# conf_data.loc[2, '2020']

###### COMPUTE GROWTH-FACTOR AND MAX CASES, MAX DATE FOR CONFIRMED CASES

In [183]:
conf_data['max_date'] = ''
conf_data['growth_factor'] = np.nan
conf_data['max_cases'] = np.nan

for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]['Cases'].max()
    if not pd.notna(max_cases):
        continue
    max_date = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long][conf_data['Cases'] == max_cases]['Date']
    
    max_date_parsed = datetime.strptime(max_date.values[0], '%m/%d/%Y')
    four_days = max_date_parsed - timedelta(days = 4)
    four_days = four_days.strftime('%-m/%d/%Y')
    
    try:
        four_days_cases = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]\
                         .loc[conf_data['Date'] == four_days]['Cases'].values[0]
        if max_cases > four_days_cases and four_days_cases != 0:
            rate = max_cases / four_days_cases 
            growth_factor = rate**0.25
        else:
            growth_factor = 1
    
    except IndexError:
        growth_factor = 1
        
    indices = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long].index.tolist()
    for index in indices:
        conf_data.at[index, 'max_date'] = max_date.values[0] 
        conf_data.at[index, 'growth_factor'] = growth_factor
        conf_data.at[index, 'max_cases'] = max_cases

/Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [187]:
death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]\
                         .loc[death_data['Date'] == four_days]['Cases']

Series([], Name: Cases, dtype: int64)

###### COMPUTE GROWTH-FACTOR AND MAX CASES, MAX DATE FOR DEATH CASES

In [188]:
death_data['max_date'] = ''
death_data['growth_factor'] = np.nan
death_data['max_cases'] = np.nan

for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]['Cases'].max()
    if not pd.notna(max_cases):
        continue
    max_date = death_data.loc[death_data['Lat'] == lat]\
                        .loc[death_data['Long'] == long][death_data['Cases'] == max_cases]['Date']
    
    max_date_parsed = datetime.strptime(max_date.values[0], '%m/%d/%Y')
    four_days = max_date_parsed - timedelta(days = 4)
    four_days = four_days.strftime('%-m/%d/%Y')
    
    try:
        four_days_cases = death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]\
                         .loc[death_data['Date'] == four_days]['Cases'].values[0]
        if max_cases > four_days_cases and four_days_cases != 0:
            rate = max_cases / four_days_cases 
            growth_factor = rate**0.25
        else:
            growth_factor = 1
    
    except IndexError:
        growth_factor = 1
    
    indices = death_data.loc[death_data['Lat'] == lat]\
                        .loc[death_data['Long'] == long].index.tolist()
    for index in indices:
        death_data.at[index, 'max_date'] = max_date.values[0] 
        death_data.at[index, 'growth_factor'] = growth_factor
        death_data.at[index, 'max_cases'] = max_cases


/Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


24


In [226]:
"""
Somehow there are 580 entries in death_cont that are not in coords
resulting in NaN values for growth factor, we will remove them for the meanwhile
"""
death_data = death_data.dropna()

'\nSomehow there are 580 entries in death_cont that are not in coords\nresulting in NaN values for growth factor, we will remove them for the meanwhile\n'

###### Add 5% interval and averages

In [ ]:
conf_data['5% date'] = ''
conf_data['avg_interval_tmp'] = np.nan
conf_data['avg_interval_RH'] = np.nan

In [ ]:
for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = int(conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]['max_cases'].values[0])
    if not pd.notna(max_cases):
        continue
    five_prcnt = int(0.05 * max_cases)
    
    five_prcnt_date = conf_data.loc[conf_data['Lat'] == lat]\
                            .loc[conf_data['Long'] == long][conf_data['Cases'] >= five_prcnt]['Date'].min()
    
    max_date = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]['max_date'].values[0]
    
    interval = conf_data.loc[conf_data['Lat'] == lat]\
                            .loc[conf_data['Long'] == long][(conf_data['Date'] >= five_prcnt_date)\
                                                            & (conf_data['Date'] <= max_date)]
    indices = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long].index.tolist()
    for index in indices:
        conf_data.at[index, '5% date'] = five_prcnt_date 
        conf_data.at[index, 'avg_interval_tmp'] = interval['avg_tmp'].mean()
        conf_data.at[index, 'avg_interval_RH'] = interval['avg_RH'].mean()

###### Sanity checks
Manual heuristic comparison of selected samples from the dataset
We compared 5 randomly selected data entries as follows:
1. lat and lan on google map
2. Daily information with Aeris API
3. Daily information with a third party climate source - https://www.worldweatheronline.com/
4. monthly information with a third party data source (This will only be an approximation) https://www.timeanddate.com/weather/israel/tel-aviv/climate

Note that coordinates are in decimal representation

###### Entries for sanity check:
              Lat	Long	avg_tmp	avg_RH	avg_wind	avg_precip	avg_m_tmp	avg_m_RH	avg_m_wind avg_m_precip
- Israel, 03/01/20, 31	35	15.3	68	23	0	14.99310345	74.5862069	13.11034483	0
    * Google maps
        - Checks good at Dimona
    * Check API
        - Checks good at Ben Gurion Air Port
    * Third party daily
        - no Ben gurion only Tel aviv
        - Everything seems good except percipitation
    * Third party monthly
         - Everything seems good except percipitation
- Afghanistan, 03/21/20 33	65	18.7	41	2.4	0	13.71315789	47.07894737	5.684210526	0.855526316
    * Google maps
        - On the mountains 200km south west of Kabul (middle of country)
    * Check API
        - Checks good at kandahar intl ar
    * Third party daily
         - Checks good, wind seems to much
    * Third party monthly
         - tmp ~3c less
         - RH ~10% more
         - m wind 5.6 instead of 14 on third part
         - m precip 0.85 instead of 3.8 in third party
- Queensland Australia, 02/03/20 -28.0167	153.4	25.6	78	12.8	0	24.05	75.55263158	16.66842105	0
    * Google maps
        - southport, in the south eastern corner of Queensland (metropolitan, but not the capital)
    * Check API
        - Checks good at the same town
    * Third party daily
        - Checks good
    * Third party monthly
        - Checks good expect precipitation
- Fiji, 03/06/20 -17.7134	178.065	26.6	90	3.2	0	26.84473684	85.47368421	5.871052632	0
    * Google maps
        - The the northen third of Fiji
    * Check API
        - Checks good for Nausori (60km south east)
    * Third party daily
        - Checks good exepct wind that seems week and precipitation that should be a little
    * Third party monthly
        - seems good exepct for precipitation
- North Dakota, US 03/12/20 47.5289	-99.784	-0.2	73	31.5	0	-4.947368421	80.52631579	18.72368421	0
    * Google maps
        - In the fields around the middle of the state
    * Check API
         - Checks good for harvey (10km north west)
    * Third party daily
        - 
    * Third party monthly
        - checks good except precipitation

After comparing the data we conclude it is relaibale except for precpitation that will be hence forth ignored.

In [35]:
data = pd.read_csv('../augmented_datasets/tableau_conf_data.csv')
RADIUS = 350

coords = [(31, 35), (33, 65), (-28.0167, 153.4), (-17.7134, 178.065), (47.5289, -99.784)] 
dates = ['03/01/2020', '03/21/2020', '03/02/2020', '03/06/2020', '03/12/2020']

for i in range(5):
    lat = coords[i][0]
    long = coords[i][1]
    start_time = dates[i]
    end_time = dates[i]
    location = data.loc[data['Lat'] == lat]\
                    .loc[data['Long'] == long]\
                     [['Country_Region','Province_State']]
    country = location.iloc[0][0]
    province = location.iloc[0][1]
    url = 'https://api.aerisapi.com/observations/summary/closest?p={0},{1}&from={2}&to={3}&radius={4}miles&plimit=31&limit=1&'\
          'client_id={5}&client_secret={6}'.format(lat, long, start_time, end_time, RADIUS, client_id, client_secret)
    print('collecting for {0} location {1}, {2}\n{3}'.format(i, province, country, url))
    request = urllib.request.urlopen(url)
    response = request.read()
    json_ = json.loads(response)
    if json_['success']: 
        print('success')
        try:
            for day in json_['response'][0]['periods']:
                print('Location: {0}, {1}'.format(json_['response'][0]['place']['name'], json_['response'][0]['place']['city']))
                print('avg tmp: {0}'.format(day['summary']['temp']['avgC']))
                print('avg RH: {0}'.format(day['summary']['rh']['avg']))
                print('avg wind: {0}'.format(day['summary']['wind']['avgKPH']))
                print('avg precip: {0}\n'.format(day['summary']['precip']['totalMM']))
        except IndexError:
            if json_['error']:
                msg = 'No data found for {0}, {1} at {2}\n{3}\r\n'.format(country, province, date, json_['error'])
                print(msg)
        except Exception as e:
            raise e
            msg = 'Unknowen error\n{0}\r\n'.format(json_['error'])
            print(msg)
        finally:
            request.close()


collecting for 0 location 0, Israel
https://api.aerisapi.com/observations/summary/closest?p=31,35&from=03/01/2020&to=03/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
Location: ben-gurion(civ/m, ben-gurion(civ/m
avg tmp: 15.3
avg RH: 68
avg wind: 23
avg precip: 0

collecting for 1 location 0, Afghanistan
https://api.aerisapi.com/observations/summary/closest?p=33,65&from=03/21/2020&to=03/21/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
Location: kandahar intl ar, kandahar intl ar
avg tmp: 18.7
avg RH: 41
avg wind: 2.4
avg precip: 0

collecting for 2 location Queensland, Australia
https://api.aerisapi.com/observations/summary/closest?p=-28.0167,153.4&from=03/02/2020&to=03/02/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
Locati

In [ ]:
######################
# SAVE COMPUTED DATA #
######################

data.to_csv('../augmented_datasets/tableau_data0403.csv')
# conf_data.to_csv('../augmented_datasets/tableau_conf_data0403.csv')
# death_data.to_csv('../augmented_datasets/tableau_death_data.csv')

###### Manually add weather data, GDP, Urbanisation to GF dataframe (this is to be fixed)

In [1]:
GF_PATH = '../augmented_datasets/tableau0403_gf.csv'
GDP_CSV = '../augmented_datasets/tableau_conf_augmented_gdp_urban.csv'

In [30]:
gf_df

,Lat,Long,Country_Region,Province_State,GFQ1,GFQ2,GFQ3,Last relevant date,5%_Date,First_7,...,03/25/2020,03/26/2020,03/27/2020,03/28/2020,03/29/2020,03/30/2020,03/31/2020,04/01/2020,04/02/2020,last relevant date
0,-41.454500,145.970700,Australia,Tasmania,0.000000,0.06450,0.305600,NaN,03/13/2020,1.000000,...,36.0,47.0,47.0,62.0,66.0,66.0,69.0,69.0,72.0,04/02/2020
1,-40.900600,174.886000,New Zealand,NaN,0.125700,0.22550,0.333300,NaN,03/21/2020,6.076923,...,205.0,283.0,368.0,451.0,514.0,589.0,647.0,708.0,797.0,04/02/2020
2,-38.416100,-63.616700,Argentina,NaN,0.100700,0.17330,0.285400,NaN,03/17/2020,3.426471,...,387.0,502.0,589.0,690.0,745.0,820.0,1054.0,1054.0,1133.0,04/02/2020
3,-37.813600,144.963100,Australia,Victoria,0.068900,0.12260,0.219500,NaN,03/15/2020,3.017544,...,466.0,520.0,574.0,685.0,769.0,821.0,917.0,968.0,1036.0,04/02/2020
4,-35.675100,-71.543000,Chile,NaN,0.123100,0.18040,0.235900,NaN,03/17/2020,2.711443,...,1142.0,1306.0,1610.0,1909.0,2139.0,2449.0,2738.0,3031.0,3404.0,04/02/2020
5,-35.473500,149.012400,Australia,Australian Capital Territory,0.039275,0.15750,0.464750,NaN,03/20/2020,7.833333,...,39.0,53.0,62.0,71.0,77.0,78.0,80.0,84.0,87.0,04/02/2020
6,-34.928500,138.600700,Australia,South Australia,0.049900,0.12590,0.291925,NaN,03/14/2020,1.631579,...,170.0,235.0,257.0,287.0,299.0,305.0,337.0,367.0,367.0,04/02/2020
7,-33.868800,151.209300,Australia,New South Wales,0.121100,0.15260,0.231600,NaN,03/15/2020,2.253731,...,1029.0,1219.0,1405.0,1617.0,1791.0,2032.0,2032.0,2182.0,2298.0,04/02/2020
8,-32.522800,-55.765800,Uruguay,NaN,0.035500,0.14810,0.189900,NaN,03/17/2020,4.586207,...,189.0,217.0,238.0,274.0,304.0,310.0,338.0,338.0,350.0,04/02/2020
9,-31.950500,115.860500,Australia,Western Australia,0.084450,0.15410,0.302500,NaN,03/16/2020,3.285714,...,175.0,231.0,231.0,278.0,311.0,355.0,364.0,392.0,400.0,04/02/2020


In [4]:
gf_df = pd.read_csv(GF_PATH)
gdp_df = pd.read_csv(GDP_CSV)

data = gf_df.copy()
states = data['Province_State'].unique().tolist()
dates = data['Date'].unique().tolist()
coords = data.groupby(['Lat', 'Long']).size().index.tolist()
data['avg_tmp'] = np.nan
data['avg_RH'] = np.nan
data['avg_wind'] = np.nan
data['avg_precip'] = np.nan
data['avg_m_tmp'] = np.nan
data['avg_m_RH'] = np.nan
data['avg_m_wind'] = np.nan
data['avg_m_precip'] = np.nan

In [27]:
data = gf_df.copy()
coords = list(zip(gf_df['Lat'].values.tolist(), gf_df['Long'].values.tolist()))
RADIUS = 350
INTERVAL = 10
q = 0

date_range = pd.date_range('2020-01-22', periods=80, freq='D')
start_time = date_range[0].strftime('%m/%d/%Y')

with open('weather_logs/tableau_log{0}.txt'.format(datetime.now().strftime('%d%m%Y')), 'w') as log:
    msg = 'Start date: {0}\nEnd date: {1}\nInterval:{2}'.format(start_time, date_range[-1].strftime('%m/%d/%Y'), INTERVAL)
    print(msg)

    for coord in coords:
        q += 1
        lat = coord[0]
        long = coord[1]
        location = data.loc[data['Lat'] == lat]\
                        .loc[data['Long'] == long]\
                         [['Country_Region','Province_State']]
        country = location.iloc[0][0]
        province = location.iloc[0][1]
        msg = 'collecting for {0} location {1}, {2}'.format(q, province, country)
        print(msg)
        _ = log.write(msg+'\n')
        for date_ in date_range[INTERVAL:][::INTERVAL]:
            end_time = date_.strftime('%m/%d/%Y')    
            url = 'https://api.aerisapi.com/observations/summary/closest?p={0},{1}&from={2}&to={3}&radius={4}miles&plimit=31&limit=1&'\
                  'client_id={5}&client_secret={6}'.format(lat, long, start_time, end_time, RADIUS, client_id, client_secret)
            msg = 'interval: {0}-{1}\n{2}\n'.format(start_time, end_time, url)
            print(msg)
            _ = log.write(msg+'\n')
            request = urllib.request.urlopen(url)
            response = request.read()
            json_ = json.loads(response)
            if json_['success']: 
                print('success')
                _ = log.write('success\n')
                try:
                    for day in json_['response'][0]['periods']:
                        date = datetime.strptime(str(day['summary']['ymd']), '%Y%m%d').strftime("%-m/%-d/%Y")
                        _ = log.write(date + '\n')
                        for index in data.loc[data['Date'] == date]\
                                         .loc[data['Lat'] == coord[0]]\
                                         .loc[data['Long'] == coord[1]].index:
                            data.at[index, 'avg_d_tmp'] = day['summary']['temp']['avgC']
                            data.at[index, 'avg_d_RH'] = day['summary']['rh']['avg']
                            data.at[index, 'avg_d_wind'] = day['summary']['wind']['avgKPH']
                            data.at[index, 'avg_d_precip'] = day['summary']['precip']['totalMM']
                except IndexError:
                    if json_['error']:
                        msg = 'No data found for {0}, {1} at {2}\n{3}\r\n'.format(country, province, date, json_['error'])
                        print(msg)
                        _ = log.write(msg)
                except Exception as e:
                    msg = 'Unknowen error\n{0}\r\n'.format(json_['error'])
                    raise e
                    print(msg)
                    _ = log.write(msg)
                finally:
                    request.close()
        start_time = date_ + timedelta(days=1)
        start_time = start_time.strftime('%m/%d/%Y')

Start date: 01/22/2020
End date: 04/10/2020
Interval:10
collecting for 1 location Tasmania, Australia
interval: 01/22/2020-02/01/2020
https://api.aerisapi.com/observations/summary/closest?p=-41.4545,145.9707&from=01/22/2020&to=02/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM

success


KeyError: 'Date'